In [38]:
import numpy as np
import itertools
import random

In [143]:
N = 3

def random_state():
	return tuple(random.choice([0,1,2]) for _ in range(N**2))

def from_string(s):
	m = {
		"_": 0,
		"X": 1,
		"O": 2,
	}
	return tuple(m[c] for c in s.strip().replace("\n", ""))


from_string("""
_OX
__X
O_X
""")

(0, 2, 1, 0, 0, 1, 2, 0, 1)

In [57]:
tiles = "_░█"
def viz(s):
	for i, f in enumerate(s):
		print(tiles[f]*2, end="")
		if not (i+1) % N:
			print()


viz(random_state())

__██░░
██____
░░__██


In [62]:
def unpack_board(s):
	return np.resize(s, (N, N))

def pack_board(board):
	return tuple(np.resize(board, (N**2,)).tolist())

def invert_board(s):
	return tuple(map(lambda col: -col % 3, s))

def generate_equivalent_boards(s):
	boards = {s}

	# Mirror
	board = unpack_board(s)
	board_flipped = np.flipud(board)
	boards.add(pack_board(board_flipped))
	
	# Rotations
	for board in [board, board_flipped]:
		for _ in range(3):
			board = np.rot90(board)
			boards.add(pack_board(board))

	# # Invert colors
	# inv = set()
	# for s in boards:
	# 	inv.add(invert_board(s))
	# boards.update(inv)

	# Sort
	boards = sorted(boards)
		
	return boards


for b in generate_equivalent_boards(random_state()):
	viz(b)
	print()

__░░░░
░░██__
██__░░

__░░██
░░██__
░░__░░

░░__░░
__██░░
██░░__

░░__░░
░░██__
__░░██

░░__██
__██░░
░░░░__

░░░░__
__██░░
░░__██

██__░░
░░██__
__░░░░

██░░__
__██░░
░░__░░



In [17]:
3 ** (N**2)

19683

## Serialization

In [64]:
def save_normalized(normalized, fname):
	with open(fname, "w") as f:
		for k, v in normalized.items():
			if k == v:
				continue
			print("{} {}".format(
				int("".join(str(c) for c in k), 3),
				int("".join(str(c) for c in v), 3)
			), file=f)

def numberToBase(n, b):
	# https://stackoverflow.com/questions/2267362/how-to-convert-an-integer-to-a-string-in-any-base
    digits = []
    while n:
        digits.append(int(n % b))
        n //= b
    while len(digits) < N**2:
        digits.append(0)
    return tuple(digits[::-1])

def load_normalized(fname):
	normalized = {}
	with open(fname) as f:
		for line in f:
			k, v = line.split(" ")
			normalized[numberToBase(int(k), 3)] = numberToBase(int(v), 3)
	return normalized


In [65]:
normalized = {}  # state -> normalized_state

for i, s in enumerate(itertools.product(range(3), repeat=N**2)):
	if not i % 1000000:
		print(i, s)
	if s in normalized:
		continue
	boards = generate_equivalent_boards(s)
	for b in boards:
		normalized[b] = boards[0]

normalized

0 (0, 0, 0, 0, 0, 0, 0, 0, 0)


{(0, 0, 0, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1): (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 1, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (1, 0, 0, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 0, 0, 2): (0, 0, 0, 0, 0, 0, 0, 0, 2),
 (0, 0, 0, 0, 0, 0, 2, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 2),
 (0, 0, 2, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 2),
 (2, 0, 0, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 2),
 (0, 0, 0, 0, 0, 0, 0, 1, 0): (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 1, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 1, 0, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 0, 1, 1): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 0, 1, 1, 0): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 1, 0, 0, 1): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 1, 0, 0, 1, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 1

In [66]:
save_normalized(normalized, fname=f"normalized-{N}.txt")

In [67]:
normalized = load_normalized(f"normalized-{N}.txt")
normalized


{(0, 0, 0, 0, 0, 0, 1, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (1, 0, 0, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 2, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 2),
 (0, 0, 2, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 2),
 (2, 0, 0, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 0, 2),
 (0, 0, 0, 0, 0, 1, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 1, 0, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 1, 1, 0): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 1, 0, 0, 1): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 1, 0, 0, 1, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 1, 0, 0, 1, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 1, 1, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (1, 0, 0, 1, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (1, 1, 0, 0, 0, 0, 0, 0, 0): (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 0, 2, 1, 0): (0, 0, 0, 0, 0, 0, 0, 1, 2

In [68]:
def normalize_state(s):
	if s not in normalized:
		return s
	return normalized[s]

## Moves

In [150]:
def place(s, col, row, color):
	return tuple(
		c if (i != N*row+col) else color
		for i, c in enumerate(s)
	)

def flip_row(s, row):
	return tuple(
		(-c % 3) if (i // N == row) else c
		for i, c in enumerate(s)
	)

def flip_column(s, col):
	return tuple(
		(-c % 3) if (i % N == col) else c
		for i, c in enumerate(s)
	)

def insert_left(s, row, color):
	new = (
		s[:N*row] 
		+ (color,) + s[N*row:N*(row+1)-1] 
		+ s[N*(row+1):]
	)
	return tuple(new)

def insert_right(s, row, color):
	new = (
		s[:N*row] 
		+ s[N*row+1:N*(row+1)] + (color,)
		+ s[N*(row+1):]
	)
	return tuple(new)

def insert_top(s, col, color):
	new = []
	for i, c in enumerate(s):
		if i % N != col:
			new.append(c)
		elif i < N:
			new.append(color)
		else:
			new.append(s[i-N])
	return tuple(new)

def insert_bottom(s, col, color):
	new = []
	for i, c in enumerate(s):
		if i % N != col:
			new.append(c)
		elif i >= N*(N-1):
			new.append(color)
		else:
			new.append(s[i+N])
	return tuple(new)


moves = {
	"p": place,
	"fr": flip_row,
	"fc": flip_column,
	"il": insert_left,
	"ir": insert_right,
	"it": insert_top,
	"ib": insert_bottom,
}


s = random_state()
viz(s)
print()
viz(flip_row(s, 1))
print()
viz(flip_column(s, 1))
print()
viz(insert_left(s, 1, 2))
print()
viz(insert_right(s, 1, 2))
print()
viz(insert_top(s, 1, 2))
print()
viz(insert_bottom(s, 1, 2))


██████
██__░░
░░████

██████
░░__██
░░████

██░░██
██__░░
░░░░██

██████
████__
░░████

██████
__░░██
░░████

██████
████░░
░░__██

██__██
████░░
░░████


In [132]:
def get_winners(s):
    winners = set()
    # Rows
    for row in range(N):
        v = s[row*N]
        if v == 0:
            continue
        for i in range(row*N+1, (row+1)*N):
            if s[i] != v:
                break
        else:
            winners.add(v)

    # Columns
    for col in range(N):
        v = s[col]
        if v == 0:
            continue
        for i in range(1, N):
            if s[col + i*N] != v:
                break
        else:
            winners.add(v)

    return winners

s = random_state()
viz(s)
get_winners(s)


██░░__
██__██
██░░██


{2}

In [147]:
def iterate_moves(s):
	for move in moves:
		# Place
		if move == "p":
			for col, row, color in itertools.product(range(N), range(N), {1, 2}):
				if s[N*row+col] != 0:
					continue
				yield ((move, col, row, color), moves[move](s, col, row, color))
		# Flip
		elif move in {"fr", "fc"}:
			for i in range(N):
				yield ((move, i), moves[move](s, i))
		# Insert
		elif move in {"il", "ir", "it", "ib"}:
			for i, color in itertools.product(range(N), {1, 2}):
				yield ((move, i, color), moves[move](s, i, color))

In [ ]:
graph = {}  # state -> ((move, *args) -> state)
terminals = {}

s_init = tuple([0] * N**2)
frontier = {s_init}

while frontier:
	s = frontier.pop()

	# Check if state is terminal
	winners = get_winners(s)
	if winners:
		graph[s] = None
		terminals[s] = winners
		continue

	for move, s_next in iterate_moves(s):
		s_next = normalize_state(s_next)
		if s not in graph:
			graph[s] = {}
		graph[s][move] = s_next

		# Add unexplored
		if s_next not in graph:
			frontier.add(s_next)

______
______
______
______
______
____░░
______
__░░__
____░░
______
__░░__
░░__░░
______
______
__░░__
______
____██
░░____
______
__░░__
____██
______
░░__██
______
____░░
______
██__░░
______
░░████
______
____░░
░░██__
____██
______
____░░
░░____
____░░
████__
____██
____░░
██__░░
____██
______
░░__░░
____██
______
______
░░__░░
______
__░░__
__████
____░░
░░██__
░░__██
______
░░__██
__██__
____░░
____░░
██____
____░░
░░░░__
____░░
______
░░████
__██__
______
____░░
░░██__
____██
░░██__
__░░░░
______
██░░██
______
____██
██__░░
__██░░
____░░
██____
____██
____░░
░░░░__
__██░░
______
__░░__
░░░░░░
____██
██____
__░░░░
____░░
____░░
██__██
____██
██____
__████
____░░
██__██
__██__
____░░
░░__██
__██░░
__░░__
██__██
░░██░░
______
░░__██
░░____
____░░
__░░__
██░░__
____██
░░░░░░
__██__
______
░░████
░░__░░
______
__░░██
██__██
____██
░░____
██░░██
____██
████░░
__░░░░
____██
██░░__
██__██
______
__░░__
__░░░░
__░░██
██____
░░██░░
____██
░░____
░░__██
______
░░████
██____
____░░
__░░__

In [153]:
moves_describe = {
	"p": "{player} places {2} tile at column {0}, row {1}.",
	"fr": "{player} flips row {0}.",
	"fc": "{player} flips column {0}.",
	"il": "{player} inserts {1} tile to the left of row {0}.",
	"ir": "{player} inserts {1} tile to the right of row {0}.",
	"it": "{player} inserts {1} tile at the top of column {0}.",
	"ib": "{player} inserts {1} tile at the bottom of column {0}.",
}

def describe_move(player, move):
	m, *args = move
	return moves_describe[m].format(*args, player=player)

s = normalize_state(random_state())
viz(s)
if s in terminals:
	print("terminal state")
else:
	for m, sn in graph[s].items():
		print(describe_move("player", m))
		viz(sn)


__██░░
██__░░
██__██
player places 1 tile at column 0, row 0.
░░░░██
██____
░░████
player places 2 tile at column 0, row 0.
░░░░██
██____
██████
player places 1 tile at column 1, row 1.
__██░░
██░░░░
██__██
player places 2 tile at column 1, row 1.
__██░░
████░░
██__██
player places 1 tile at column 1, row 2.
__██░░
██__░░
██░░██
player places 2 tile at column 1, row 2.
__██░░
██__░░
██████
player flips row 0.
__░░██
██__░░
██__██
player flips row 1.
__░░██
██____
░░████
player flips row 2.
__██░░
██____
░░░░░░
player flips column 0.
__░░░░
██____
░░░░██
player flips column 1.
__░░░░
██__░░
██__██
player flips column 2.
__████
██____
████░░
player inserts 1 tile to the left of row 0.
░░__██
██__░░
██__██
player inserts 2 tile to the left of row 0.
██__██
░░__██
██__██
player inserts 1 tile to the left of row 1.
__░░██
████__
░░__██
player inserts 2 tile to the left of row 1.
__██░░
████__
██__██
player inserts 1 tile to the left of row 2.
__░░░░
██__██
░░██__
player inserts 2 tile to th